In [1]:
import pandas as pd
import os

from sql_model import *
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

In [2]:
ROOT_DIR = os.getcwd()
SECRET_DIR = os.path.join(ROOT_DIR, '.secrets')

secrets = json.load(open(os.path.join(SECRET_DIR, "secrets.json")))


engine = create_engine('mysql+pymysql://{}:{}@{}:{}/{}'.format(
    secrets['RDS_USER_ID'],
    secrets['RDS_USER_PASSWORD'],
    secrets['RDS_USER_URL'],
    secrets['RDS_PORT'],
    secrets['DATABASE_NAME']
))

In [3]:
Sessoion = sessionmaker(bind=engine)
session = Sessoion()

In [4]:
Base.metadata.create_all(engine)

C:\Users\MSI\.conda\envs\dbs\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect string value: '\\xB4\\xEB\\xC7\\xD1\\xB9\\xCE...' for column 'VARIABLE_VALUE' at row 485")
  result = self._query(query)


In [5]:
data_dir = ['category.csv', 'facility.csv', 'region.csv', 'info.csv']

In [6]:
for file_path in data_dir:
    data = pd.read_csv('./data/{}'.format(file_path), keep_default_na=False, encoding="cp949", header=None)
    if file_path == 'info.csv':
        for index in range(len(data)):
            row = Information(
                name = data.loc[index][3],
                tel = data.loc[index][4],
                fare = data.loc[index][5],
                latitude = float(data.loc[index][6]),
                longitude = float(data.loc[index][7])
            )
            row.region_id = (session.query(Region).filter(Region.code == data.loc[index][0]).all())[0].id
            row.facility_id = (session.query(Facility).filter(Facility.code == data.loc[index][1]).all())[0].id
            row.category_id = (session.query(Category).filter(Category.name == data.loc[index][2]).all())[0].id
            session.add(row)
        session.commit()
        print("{} DB CREATE".format(file_path))

    elif file_path == 'facility.csv':
        for index in range(len(data)):
            row = Facility(
                code=data.loc[index][0],
                name=data.loc[index][1]
            )
            session.add(row)
        session.commit()
        print("{} DB CREATE".format(file_path))
    
    elif file_path == 'category.csv':
        for index in range(len(data)):
            row = Category(
                name=data.loc[index][0]
            )
            session.add(row)
        session.commit()
        print("{} DB CREATE".format(file_path))

    else:
        for index in range(len(data)):
            row = Region(
                code=data.loc[index][0],
                name=data.loc[index][1]
            )

            session.add(row)
        session.commit()
        print("{} DB CREATE".format(file_path))

category.csv DB CREATE
facility.csv DB CREATE
region.csv DB CREATE
info.csv DB CREATE
